<a href="https://colab.research.google.com/github/SoohyeonB/Ewha-git/blob/main/kobert_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3.0.2
# !pip install torch==1.2.0
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

     |████████████████████████████████| 46.9 MB 46 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595741 sha256=ab230a9a5b9d025c6f09df18e3c74ab59a1c17cd320315066ab005003c959004
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.1 MB 5.5 MB/s 
     |████████████████████████████████| 2.5 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 26.2 MB/s 
     |████████████████████████████████| 895 kB 36.7 MB/s 
     |████████████████████████████████| 804.1 MB 1.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfu

In [3]:
# SKT KoBERT_hf
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-wgtlx5ku/kobert-tokenizer_851e2e67fb9c47ab8cf1eece1abaaf90
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-wgtlx5ku/kobert-tokenizer_851e2e67fb9c47ab8cf1eece1abaaf90
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=7b1cbf2a8588b0cfcbfc6513c4f2b173211db0ff4d9e3c3f0ad365666018b9d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-bpnqxej4/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#kobert
#from kobert.utils import get_tokenizer
from kobert_tokenizer import KoBERTTokenizer
#from kobert.pytorch_kobert import get_pytorch_kobert_model




#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")


#device = torch.device("cpu")

In [6]:
#BERT 모델, Vocabulary 불러오기
#bertmodel, vocab = get_pytorch_kobert_model()

bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')

TypeError: ignored

In [ ]:
# 챗봇 데이터셋
chatbot_data = pd.read_excel('/content/drive/MyDrive/KoBERT2/data (1).xlsx')

In [ ]:

# answer json 파일 열기

# import json
# from google.colab import files
# with open('/content/drive/MyDrive/KoBERT/chattest1.json') as json_file:
#   answer_data = json.load(json_file)



In [ ]:
#chatbot_data.sample(n=10)

In [ ]:
#print(answer_data)

In [ ]:
# 특수문자 제거하기
import re
def cleanText(input):
  clean = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',input)
  return clean

In [ ]:
# 네이버 맞춤법 교정기 기반 라이브러리
!pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker

def spellCheck(input):
  checked = spell_checker.check(input)
  #print(checked.as_dict())
  #print(checked.as_dict()['checked'])
  return checked.as_dict()['checked']


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-c01xj8ao
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-c01xj8ao
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=768a9918b60cda9fcd6f0b2ba01bca85a5073101b945fa91cbac128aa56be544
  Stored in directory: /tmp/pip-ephem-wheel-cache-u4fy5z63/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_방법"), '클래스'] = 0  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_시기"), '클래스'] = 1  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_이후"), '클래스'] = 2  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_화분"), '클래스'] = 3  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_흙"), '클래스'] = 4  
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_화상"), '클래스'] = 5 
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_온도"), '클래스'] = 6
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_인조조명"), '클래스'] = 7
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_장소"), '클래스'] = 8
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_햇빛"), '클래스'] = 9
chatbot_data.loc[(chatbot_data['클래스'] == "튜토리얼_방법"), '클래스'] = 10
chatbot_data.loc[(chatbot_data['클래스'] == "물주기"), '클래스'] = 11 
chatbot_data.loc[(chatbot_data['클래스'] == "물주기_저면관수"), '클래스'] = 12 
chatbot_data.loc[(chatbot_data['클래스'] == "물주기_계절"), '클래스'] = 13
chatbot_data.loc[(chatbot_data['클래스'] == "번식_잎꽂이방법"), '클래스'] = 14 
chatbot_data.loc[(chatbot_data['클래스'] == "번식_자구"), '클래스'] = 15
chatbot_data.loc[(chatbot_data['클래스'] == "번식_적심"), '클래스'] = 16
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_무름병"), '클래스'] = 17
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_썩음병"), '클래스'] = 18
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_기타"), '클래스'] = 19
chatbot_data.loc[(chatbot_data['클래스'] == "웃자람_증상"), '클래스'] = 20
chatbot_data.loc[(chatbot_data['클래스'] == "웃자람_해결"), '클래스'] = 21 
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_곰팡이"), '클래스'] = 22
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_과습"), '클래스'] = 23
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_과습여부"), '클래스'] = 24
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_마름"), '클래스'] = 25
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_원인알수없음"), '클래스'] = 26
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_탄저병"), '클래스'] = 27
chatbot_data.loc[(chatbot_data['클래스'] == "해충_기타"), '클래스'] = 28
chatbot_data.loc[(chatbot_data['클래스'] == "해충_깍지"), '클래스'] = 29
chatbot_data.loc[(chatbot_data['클래스'] == "해충_응애"), '클래스'] = 30
chatbot_data.loc[(chatbot_data['클래스'] == "해충_해충약"), '클래스'] = 31


data_list = []
for ques, label in zip(chatbot_data['질문내용'], chatbot_data['클래스'])  :
    data = []
    ques = cleanText(ques)
    #ques = spellCheck(ques)
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[5])
print(data_list[123])
print(data_list[200])

['회색곰팡이 같은데 조언을 구합니다', '22']
['잎이 물렁물렁해', '23']
['무름병인지 아닌지 모르겠네요', '24']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

2228
558


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화

#tokenizer = get_tokenizer()
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


NameError: ignored

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


NameError: ignored

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=32,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

NameError: ignored

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/35 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.506265163421631 train acc 0.09375
epoch 1 train acc 0.04261675824175824


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

...

epoch 14 test acc 0.8848128019323672


  0%|          | 0/35 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.16104677319526672 train acc 0.984375
epoch 15 train acc 0.9910714285714286


  0%|          | 0/9 [00:00<?, ?it/s]

epoch 15 test acc 0.8848128019323672


In [ ]:
# plot

# import matplotlib.pyplot as plt

# loss_train = train_history
# loss_val = test_history
# sub_train = train_history[2:12]
# sub_val = test_history[0:10]
# epochs = range(1,11)
# plt.plot(epochs, sub_train, 'g', label='Training accuracy')
# plt.plot(epochs, sub_val, 'b', label='validation accuracy')
# plt.title('Training and Validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()


In [ ]:

# loss_loss=loss_history
# epochs = range(1,16)
# plt.plot(epochs, loss_loss, 'l', label='Loss')
# plt.title('Training Loss curve')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
# import random 
# def getAnswer(index):
#   #answers=[]
#   for tg in answer_data["intents"]:
#     if tg['tag']==str(index):
#       answers = tg['answer']

#   ans = random.choice(answers)
#   return ans

In [ ]:
# # 수정본
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# def predict(predict_sentence):

#     predict_sentence = spellCheck(predict_sentence)
#     predict_sentence = cleanText(predict_sentence)

#     #print("전처리 후: "+predict_sentence)
    
#     data = [predict_sentence, '0']
#     #print(data) #['물 줄까', '0']

#     dataset_another = [data]
#     #print(dataset_another) #[['물 줄까', '0']]

#     another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    
#     test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2) 
    
#     model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)

#         valid_length= valid_length
#         label = label.long().to(device)

        
        
#         out = model(token_ids, valid_length, segment_ids)
#         tmp = out.detach().cpu().numpy()
#         maxProb = np.max(tmp[0])
#         test_eval=[]
#         idx = -1
#         for i in out:
#             logits=i
#             logits = logits.detach().cpu().numpy()

#             idx = np.argmax(logits)

#             if idx == 0:
#                 test_eval.append("곰팡이")
#             elif idx == 1:
#                 test_eval.append("물주기")
#             elif idx == 2:
#                 test_eval.append("분갈이방법")
#             elif idx == 3:
#                 test_eval.append("온도")
#             elif idx == 4:
#                 test_eval.append("웃자람")
#             elif idx == 5:
#                 test_eval.append("인조조명")
#             elif idx == 6:
#                 test_eval.append("잎의질병")
#             elif idx == 7:
#                 test_eval.append("잡초")
#             elif idx == 8:
#                 test_eval.append("튜토리얼")
#             elif idx == 9:
#                 test_eval.append("해충")
#         #print(">> 질문 카테고리: " + test_eval[0])
        
#         answer = getAnswer(idx)
#         #print(">> 답변: "+answer)

#         return answer



In [ ]:
# %cd '/content/drive/MyDrive/KoBERT/saved_model'
# load_model=torch.load('torch_pkl.pkl',map_location=torch.device('cpu'))

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/KoBERT/saved_model/new_model.pkl')


In [ ]:
# device = torch.device("cuda") 
# load_model = TheModelClass(*args, **kwargs) 
# load_model.load_state_dict(torch.load('/content/drive/MyDrive/KoBERT/saved_model/torch_model.pt')) 
# load_model.to(device)


In [ ]:
# #질문 무한반복
# end = 1
# while end == 1 :
#     sentence = input("입력 : ")
#     if sentence == "quit" :
#         break
#     print(predict(sentence))
#     print("\n")

In [ ]:
# predict("다육이 키우기")

# def chat():

#   while True:
#     inp = input("입력: ")
#     if inp.lower()=="quit":
#       break

#     print(predict(inp))

In [ ]:
#!mkdir -p saved_model
#torch.save(model,'/content/drive/MyDrive/KoBERT/saved_model/h5model.h5')

In [ ]:
#!jupyter notebook --ip=0.0.0.0 --port=8888 --allow-root

In [ ]:
# from flask import Flask, request, jsonify

# app = Flask(__name__)

# @app.route('/chat', methods=['GET','POST'])
# def chat():
#   chatInput = request.form['chatInput']
#   return jsonify(chatBotReply=predict(chatInput))

# if __name__ == '__main__':
#   #app.run(host='192.168.61.1',debug=True)
#   app.run()
# # import sys
# # sys.path.append('/content/drive/MyDrive/KoBERT/app.py')

In [ ]:
# !FLASK_ENV=development FLASK_APP=app.py flask run

In [ ]:
# !pi install flask==0.12.2
# !pip install flask-ngrok

In [ ]:
# chatBotReply=predict("다육이 키우는 방법")
# print(chatBotReply)


In [ ]:
#model.save_pretrained("")



In [ ]:
# # from flask_ngrok import run_with_ngrok
# # from flask import Flask, request, jsonify

# # app = Flask(__name__)
# # run_with_ngrok(app)
# # @app.route('/chat') #, methods=['GET','POST']
# # def chatBot():
# #   chatInput = request.form['chatInput']
# #   #return jsonify(chatBotReply=predict(chatInput))
# #   return predict(chatInput)

# # app.run()
# # # if __name__ == '__main__':
# # #   app.run()

# # flask_ngrok_example.py


# # from flask import Flask
# # from flask_ngrok import run_with_ngrok

# # app = Flask(__name__)
# # run_with_ngrok(app)  # Start ngrok when app is run

# # @app.route("/",methods=['GET','POST'])
# # def chatBot():
# #   chatInput = request.form['chatInput']
# #   #return jsonify(chatBotReply=predict(chatInput))
# #   return predict(chatInput)

# # if __name__ == '__main__':
# #     app.run()

# from flask import Flask
# from flask_ngrok import run_with_ngrok

# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run

# @app.route("/chat",methods=['GET','POST'])
# def chatBot():
#   chatInput = request.args.get('chatInput')
#   #return jsonify(chatBotReply=predict(chatInput))
#   return predict(chatInput)

# if __name__ == '__main__':
#     app.run()

In [ ]:
# # predict with load_model
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# def predict(predict_sentence):

#     predict_sentence = spellCheck(predict_sentence)
#     predict_sentence = cleanText(predict_sentence)

#     #print("전처리 후: "+predict_sentence)
    
#     data = [predict_sentence, '0']
#     #print(data) #['물 줄까', '0']

#     dataset_another = [data]
#     #print(dataset_another) #[['물 줄까', '0']]

#     another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    
#     test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2) 
    
#     load_model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)

#         valid_length= valid_length
#         label = label.long().to(device)

        
        
#         out = load_model(token_ids, valid_length, segment_ids)
#         tmp = out.detach().cpu().numpy()
#         maxProb = np.max(tmp[0])

#         test_eval=[]
#         idx = -1
#         for i in out:
#             logits=i
#             logits = logits.detach().cpu().numpy()

#             idx = np.argmax(logits)

#             if idx == 0:
#                 test_eval.append("곰팡이")
#             elif idx == 1:
#                 test_eval.append("물주기")
#             elif idx == 2:
#                 test_eval.append("분갈이방법")
#             elif idx == 3:
#                 test_eval.append("온도")
#             elif idx == 4:
#                 test_eval.append("웃자람")
#             elif idx == 5:
#                 test_eval.append("인조조명")
#             elif idx == 6:
#                 test_eval.append("잎의질병")
#             elif idx == 7:
#                 test_eval.append("잡초")
#             elif idx == 8:
#                 test_eval.append("튜토리얼")
#             elif idx == 9:
#                 test_eval.append("해충")
#         #print(">> 질문 카테고리: " + test_eval[0])
        
#         answer = getAnswer(idx)
#         #print(">> 답변: "+answer)

#         return answer

In [ ]:
!pip install kss

In [ ]:
# 문장 분리
import kss
from kss import split_sentences

s = "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습." 
for sent in kss.split_sentences(s): 
  print(sent)



회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요
다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습.
